In [147]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
import pandas as pd


Machine_model_dic = {
    "Linear" : LinearRegression(),
    "svm" : svm.SVR(),
    "predict" : RandomForestRegressor()
}

train = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/test.csv')

Dtrain = pd.DataFrame(train)
Dtest = pd.DataFrame(test)

# train 분리
# drop : columns = '?' ? 빼고
X_train = Dtrain.drop(columns=['body fat_%'])
Y_train = Dtrain['body fat_%']

# 학습시킬 데이터 
X_train['gender'] = X_train['gender'].map({'M':1,'F':0})
X_train

# 테스트할 데이터
X_test = Dtest
X_test['gender'] = X_test['gender'].map({'M':1,'F':0})


In [105]:
X_train

,age,gender,height_cm,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,23.0,1,173.7,87.0,143.0,52.9,18.0,57.0,233.0
1,24.0,1,181.6,65.0,126.0,45.1,8.3,38.0,254.0
2,36.0,0,166.5,84.0,130.0,27.4,18.9,40.0,186.0
3,22.0,0,165.8,77.0,130.0,29.7,19.0,30.0,183.0
4,61.0,0,144.7,77.0,130.0,20.7,11.8,7.0,126.0
...,...,...,...,...,...,...,...,...,...
9370,21.0,1,172.3,87.0,127.0,42.1,7.6,41.0,205.0
9371,29.0,1,167.1,83.0,142.0,38.8,23.1,51.0,199.0
9372,24.0,0,167.7,80.0,136.0,30.1,22.0,47.0,182.0
9373,22.0,0,159.1,64.0,114.0,22.3,19.1,50.0,172.0


In [106]:
Y_train

0       12.7
1        3.5
2       25.9
3       33.0
4       28.3
        ... 
9370    16.2
9371    19.3
9372    29.1
9373    27.4
9374    21.0
Name: body fat_%, Length: 9375, dtype: float64

In [148]:
model_p = {}

In [149]:
# 학습 및 평가 데이터를 DataFrame 형태로 입력받아 정규화된 데이터로 변환하는 함수 
def scale(x_train, x_test):
    columns = x_train.columns

    scaler = StandardScaler()

    # 학습 데이터의 age ~ broad jump_cm 열에 해당하는 값을 가져와 피팅 시킵니다.
    scaler.fit(x_train)

    # 피팅된 scaler를 기준으로 학습데이터를 정규화합니다.
    x_train = scaler.transform(x_train)
    # 평가 데이터도 마찬가지로 학습 데이터로 피팅된 scaler를 기준으로 정규화를 진행합니다.
    x_test = scaler.transform(x_test)

    # DataFrame 형태로 변환하여 반환합니다.
    x_train = pd.DataFrame(columns=columns, data=x_train)
    x_test = pd.DataFrame(columns=columns, data=x_test)

    return x_train, x_test

# 학습 데이터, 평가 데이터를 정의한 scale 함수에 전달하여 정규화된 학습 및 평가 데이터를 x_train, x_test에 저장합니다.
X_train, X_test = scale(X_train, X_test)



In [119]:
X_test

,age,gender,height_cm,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,0.375900,0.763072,-1.170605,1.314133,0.993000,-0.185764,0.100569,0.364193,0.446036
1,-0.870648,0.763072,0.358930,1.592985,0.448286,1.241150,-0.190398,0.504858,0.521137
2,-0.430690,0.763072,0.074365,-0.451928,-0.777322,1.926444,0.426452,1.419185,1.497447
3,0.155921,0.763072,0.252218,-0.637829,0.516375,0.612181,0.403175,0.153194,0.796506
4,1.475795,-1.310493,-0.162772,-0.544879,-0.300697,-1.828593,-2.168975,-2.800783,-3.133769
...,...,...,...,...,...,...,...,...,...
4013,-1.017300,0.763072,-0.352482,-1.102582,-0.300697,0.321166,1.555405,0.856522,-0.104703
4014,-1.017300,0.763072,0.181077,-1.288484,-0.232608,0.208515,-0.178759,1.278519,0.095566
4015,0.009268,-1.310493,-0.423623,0.849380,-0.164518,-1.133911,0.554478,-0.198470,-0.830678
4016,-1.090627,0.763072,0.311502,0.198726,1.197268,0.584018,-0.446449,0.364193,1.297178


In [120]:
X_train

,age,gender,height_cm,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,-1.017300,0.763072,0.607924,0.756429,0.856821,1.494615,0.321704,1.208186,1.071876
1,-0.943974,0.763072,1.544615,-1.288484,-0.300697,0.762383,-0.807248,-0.128137,1.597582
2,-0.064058,-1.310493,-0.245770,0.477578,-0.028340,-0.899221,0.426452,0.012529,-0.104703
3,-1.090627,-1.310493,-0.328768,-0.173076,-0.028340,-0.683307,0.438091,-0.690799,-0.179804
4,1.769101,-1.310493,-2.830565,-0.173076,-0.028340,-1.528190,-0.399894,-2.308454,-1.606719
...,...,...,...,...,...,...,...,...,...
9370,-1.163953,0.763072,0.441928,0.756429,-0.232608,0.480755,-0.888719,0.082861,0.370935
9371,-0.577342,0.763072,-0.174629,0.384627,0.788732,0.170964,0.915277,0.786189,0.220734
9372,-0.943974,-1.310493,-0.103488,0.105775,0.380196,-0.645756,0.787252,0.504858,-0.204838
9373,-1.090627,-1.310493,-1.123178,-1.381434,-1.117769,-1.377989,0.449730,0.715857,-0.455174


In [150]:
for key in Machine_model_dic.keys():
  Machine_model_dic[key].fit(X_train, Y_train)
  model_p[key] = Machine_model_dic[key].predict(X_test)


In [151]:
model_pdf = pd.DataFrame(model_p)

In [152]:
model_pdf = pd.DataFrame(model_p)
model_pdf

,Linear,svm,predict
0,21.813013,21.696685,23.201000
1,22.395865,22.647012,23.740200
2,15.693965,16.103666,15.243000
3,18.992985,19.367135,20.707000
4,38.015326,35.192692,38.176200
...,...,...,...
4013,20.012904,20.581585,19.897000
4014,19.141298,19.228833,20.929000
4015,27.898273,27.231353,27.830000
4016,19.853753,17.759322,16.726000


In [153]:
Lp = pd.DataFrame(model_pdf['Linear'])
Sp = pd.DataFrame(model_pdf['svm'])
Pp = pd.DataFrame(model_pdf['predict'])

In [154]:
submit = pd.read_csv('/content/drive/MyDrive/dataset/submit_sample.csv')
submit['predict'] = Pp

submit 

,predict
0,23.201000
1,23.740200
2,15.243000
3,20.707000
4,38.176200
...,...
4013,19.897000
4014,20.929000
4015,27.830000
4016,16.726000


In [155]:
submit.to_csv('/content/drive/MyDrive/dataset/my_submit.csv', index=False)